In [271]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

In [272]:
data=pd.read_csv("Rainfall.csv")

In [273]:
data.head()

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,rainfall,sunshine,winddirection,windspeed
0,1,1025.9,19.9,18.3,16.8,13.1,72,49,yes,9.3,80.0,26.3
1,2,1022.0,21.7,18.9,17.2,15.6,81,83,yes,0.6,50.0,15.3
2,3,1019.7,20.3,19.3,18.0,18.4,95,91,yes,0.0,40.0,14.2
3,4,1018.9,22.3,20.6,19.1,18.8,90,88,yes,1.0,50.0,16.9
4,5,1015.9,21.3,20.7,20.2,19.9,95,81,yes,0.0,40.0,13.7


In [274]:
data.columns=[a.strip() for a in data.columns]

In [275]:
data.isna().sum()

day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
rainfall         0
sunshine         0
winddirection    1
windspeed        1
dtype: int64

In [276]:
data.dropna(inplace=True)

In [277]:
data.rainfall.value_counts()

rainfall
yes    248
no     117
Name: count, dtype: int64

In [278]:
data.rainfall=data.rainfall.replace({'yes':1,'no':0})

C:\Users\vaibh\AppData\Local\Temp\ipykernel_17568\1609939009.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.rainfall=data.rainfall.replace({'yes':1,'no':0})


In [279]:
data.drop('day',axis=1,inplace=True)

In [280]:
data.head()

,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,rainfall,sunshine,winddirection,windspeed
0,1025.9,19.9,18.3,16.8,13.1,72,49,1,9.3,80.0,26.3
1,1022.0,21.7,18.9,17.2,15.6,81,83,1,0.6,50.0,15.3
2,1019.7,20.3,19.3,18.0,18.4,95,91,1,0.0,40.0,14.2
3,1018.9,22.3,20.6,19.1,18.8,90,88,1,1.0,50.0,16.9
4,1015.9,21.3,20.7,20.2,19.9,95,81,1,0.0,40.0,13.7


In [281]:
data.groupby('rainfall').median()

,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
rainfall,,,,,,,,,,
0,1013.10,29.10,25.50,23.7,20.9,75.0,49.0,8.8,70.0,16.7
1,1012.75,27.55,25.35,23.7,22.3,84.0,84.0,1.5,70.0,21.9


In [282]:
x=data.drop('rainfall',axis=1).values
y=data.rainfall.values

In [283]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [284]:
x_norm=scaler.fit_transform(x)

In [285]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=500)

In [286]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(knn, x, y, cv=5)
print(f"KNN Cross-Validation Scores: {cv_scores}")
print(f"Mean CV Score: {np.mean(cv_scores)}")

KNN Cross-Validation Scores: [0.84931507 0.73972603 0.7260274  0.78082192 0.78082192]
Mean CV Score: 0.7753424657534247


In [287]:
x_train_norm=scaler.fit_transform(x_train)
x_test_norm=scaler.fit_transform(x_test)

In [288]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors': [3, 5, 7, 9]}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(x_train_norm, y_train)

print(f"Best Parameters: {grid_search.best_params_}")
knn = grid_search.best_estimator_

Best Parameters: {'n_neighbors': 9}


In [289]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train_norm,y_train)

KNeighborsClassifier()

In [290]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,knn.predict(x_test_norm))

0.782608695652174

In [291]:
confusion_matrix(y_test,knn.predict(x_test_norm))

array([[17,  9],
       [11, 55]], dtype=int64)

In [292]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(random_state=10)
lr.fit(x_train_norm,y_train)

LogisticRegression(random_state=10)

In [293]:
accuracy_score(y_test,lr.predict(x_test_norm))

0.8152173913043478

In [294]:
confusion_matrix(y_test,lr.predict(x_test_norm))

array([[18,  8],
       [ 9, 57]], dtype=int64)

In [295]:
from sklearn.svm import SVC
svc=SVC(kernel='linear')
svc.fit(x_train_norm,y_train)

SVC(kernel='linear')

In [296]:
accuracy_score(y_test,svc.predict(x_test_norm))

0.8152173913043478

In [297]:
confusion_matrix(y_test,svc.predict(x_test_norm))

array([[18,  8],
       [ 9, 57]], dtype=int64)

In [298]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB()
nb.fit(x_train_norm,y_train)

GaussianNB()

In [299]:
print(accuracy_score(y_test,nb.predict(x_test_norm)))
confusion_matrix(y_test,nb.predict(x_test_norm))

0.75


array([[17,  9],
       [14, 52]], dtype=int64)

In [300]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=100,criterion='entropy')
rfc.fit(x_train_norm,y_train)

RandomForestClassifier(criterion='entropy')

In [301]:
import shap
explainer = shap.TreeExplainer(rfc)
shap_values = explainer.shap_values(x_test_norm, check_additivity=False)
shap.summary_plot(shap_values, x_test_norm, feature_names=data.columns[:-1])


<Figure size 1150x660 with 2 Axes>

In [302]:
importances = rfc.feature_importances_
indices = np.argsort(importances)[::-1]
plt.figure()
plt.title("Feature Importances")
plt.bar(range(x.shape[1]), importances[indices], align="center")
plt.xticks(range(x.shape[1]), data.columns[:-1][indices], rotation=90)
plt.xlim([-1, x.shape[1]])
plt.show()

<Figure size 640x480 with 1 Axes>

In [303]:
print(accuracy_score(y_test,rfc.predict(x_test_norm)))
confusion_matrix(y_test,rfc.predict(x_test_norm))

0.7608695652173914


array([[17,  9],
       [13, 53]], dtype=int64)

In [304]:
ann=tf.keras.Sequential()

In [305]:
ann.add(tf.keras.layers.Dense(units=7,activation='relu'))
ann.add(tf.keras.layers.Dense(units=7,activation='relu'))

In [306]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [307]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
ann.fit(x_train_norm,y_train,batch_size=32,epochs=30)

Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3470 - loss: 0.7810  
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3478 - loss: 0.7302 
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5448 - loss: 0.6884 
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6941 - loss: 0.6489 
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6631 - loss: 0.6300 
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.6643 - loss: 0.6091
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - accuracy: 0.6521 - loss: 0.6019
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - accuracy: 0.6520 - loss: 0.6018
Epoch 9/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - accuracy: 0.6722 - loss: 0.5714
Epoch 10/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - accuracy: 0.6677 - loss: 0.5710
Epoch 11/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6797 - loss: 0.5542 
Epoch 12/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6702 - loss: 0.5

In [308]:
test_loss, test_accuracy = ann.evaluate(x_test_norm, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.4f}")

y_pred = ann.predict(x_test_norm)
y_pred_classes = (y_pred > 0.5).astype(int).flatten()

from sklearn.metrics import accuracy_score, confusion_matrix
print(f"Accuracy Score: {accuracy_score(y_test, y_pred_classes):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))

3/3 - 0s - 33ms/step - accuracy: 0.8043 - loss: 0.4483
Test Accuracy: 0.8043
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Accuracy Score: 0.8043
Confusion Matrix:
[[15 11]
 [ 7 59]]


In [309]:
def plot_and_save_cm(model, x_test, y_test, model_name):
    # Generate predictions
    if model_name == "ANN":
        y_pred = (model.predict(x_test) > 0.5).astype(int).flatten()
    else:
        y_pred = model.predict(x_test)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(10,7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['No Rain', 'Rain'], 
                yticklabels=['No Rain', 'Rain'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix for {model_name}')
    plt.savefig(f'{model_name}_confusion_matrix.png')
    plt.show()

# Plot confusion matrices for all models
plot_and_save_cm(knn, x_test_norm, y_test, "KNN")
plot_and_save_cm(lr, x_test_norm, y_test, "Logistic Regression")
plot_and_save_cm(svc, x_test_norm, y_test, "SVM")
plot_and_save_cm(nb, x_test_norm, y_test, "Naive Bayes")
plot_and_save_cm(rfc, x_test_norm, y_test, "Random Forest")
plot_and_save_cm(ann, x_test_norm, y_test, "ANN")


<Figure size 1000x700 with 2 Axes>

<Figure size 1000x700 with 2 Axes>

<Figure size 1000x700 with 2 Axes>

<Figure size 1000x700 with 2 Axes>

<Figure size 1000x700 with 2 Axes>

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step


<Figure size 1000x700 with 2 Axes>

In [310]:
import gradio as gr

def predict_rainfall(humidity, temperature, windspeed):
    # Ensure values are within reasonable ranges
    humidity = min(max(humidity, 0), 100)
    temperature = min(max(temperature, -10), 50)
    windspeed = min(max(windspeed, 0), 100)

    default_values = [0.5] * 7
    input_data = np.array([[humidity, temperature, windspeed] + default_values])
    
    input_data_norm = scaler.transform(input_data)
    
    knn_pred = knn.predict(input_data_norm)[0]
    lr_pred = lr.predict(input_data_norm)[0]
    svc_pred = svc.predict(input_data_norm)[0]
    nb_pred = nb.predict(input_data_norm)[0]
    rfc_pred = rfc.predict(input_data_norm)[0]
    ann_pred = (ann.predict(input_data_norm) > 0.5).astype(int)[0][0]

    return {
        "KNN Prediction": "Rain" if knn_pred == 1 else "No Rain",
        "Logistic Regression Prediction": "Rain" if lr_pred == 1 else "No Rain",
        "SVM Prediction": "Rain" if svc_pred == 1 else "No Rain",
        "Naive Bayes Prediction": "Rain" if nb_pred == 1 else "No Rain",
        "Random Forest Prediction": "Rain" if rfc_pred == 1 else "No Rain",
        "ANN Prediction": "Rain" if ann_pred == 1 else "No Rain",
    }



inputs = [
    gr.Slider(0, 100, step=1, label="Humidity", interactive=True, elem_id="humidity-input"),
    gr.Slider(-10, 50, step=0.5, label="Temperature (°C)", interactive=True, elem_id="temp-input"),
    gr.Slider(0, 100, step=1, label="Windspeed (km/h)", interactive=True, elem_id="wind-input")
]

outputs = gr.JSON(label="Model Predictions", elem_id="output")

gr.Interface(
    fn=predict_rainfall, 
    inputs=inputs, 
    outputs=outputs, 
    title="Rainfall Prediction Models",
    description="Enter the weather conditions to predict whether rainfall is likely based on different machine learning models. The interface also shows performance metrics and visualizations of the models.",
    theme="compact",
    css="""
    body { 
        background-color: #e6f7ff; 
        font-family: 'Arial', sans-serif; 
    }
    .gr-slider input {
        accent-color: lightblue;
    }
    #humidity-input input, #temp-input input, #wind-input input {
        border: 2px solid #4CAF50;
        padding: 10px;
        border-radius: 5px;
        color : lightblue;
    }
    .gr-button-primary {
        background-color: #4CAF50;
        border-color: #4CAF50;
        color: white;
        padding: 10px;
        border-radius: 5px;
    }
    #output {
        border: 2px solid #4CAF50;
        padding: 10px;
        border-radius: 5px;
        font-size: 1.2em;
        background-color: #e6ffe6;
        color: #333;
    }
    h1 {
        color: #4CAF50;
    }
    .gradio-container {
        padding: 20px;
        box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
        border-radius: 10px;
    }
"""

).launch()


C:\Users\vaibh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\gradio\blocks.py:982: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-66dd779b-0ae2390920b27f6d2de99da4;8baacf5d-6660-43d9-bbbf-feb138b179bd)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [311]:
import joblib

# Save models
joblib.dump(knn, 'knn_model.pkl')
joblib.dump(lr, 'lr_model.pkl')
joblib.dump(svc, 'svc_model.pkl')
joblib.dump(nb, 'nb_model.pkl')
joblib.dump(rfc, 'rfc_model.pkl')
ann.save('ann_model.h5')

# Load models (add this at the start of your script)
knn = joblib.load('knn_model.pkl')
lr = joblib.load('lr_model.pkl')
svc = joblib.load('svc_model.pkl')
nb = joblib.load('nb_model.pkl')
rfc = joblib.load('rfc_model.pkl')
ann = tf.keras.models.load_model('ann_model.h5')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
